In [6]:
from bs4 import BeautifulSoup
import pandas as pd
import sklearn

In [7]:
# Load the CSV file into a DataFrame
df = pd.read_csv('6n.csv')

# Inspect the first few rows and columns
print(df.head())
print(df.columns)

                              DOI  \
0       10.47363/jaicc/2024(3)347   
1  10.70715/jitcai.2024.v1.i1.004   
2            10.2139/ssrn.4782875   
3      10.9781/ijimai.2024.02.006   
4            10.2139/ssrn.4614223   

                                               Title  \
0  Impact of AI and GenAI on Healthcare: Security...   
1  The Impact of GenAI on Student Engagement and ...   
2  Knowledge Management Perspective of GenAI (GenAI)   
3  GenAI in Product Design Education: Navigating ...   
4  GenAI Against Humanity: Nefarious Applications...   

                                            Abstract  
0  <jats:p>This paper aims to understand the impa...  
1  <jats:p>The rapid adoption of AI (AI) in highe...  
2                                                NaN  
3                                                NaN  
4                                                NaN  
Index(['DOI', 'Title', 'Abstract'], dtype='object')


In [8]:
# Remove rows where 'Abstract' column has the value 'N/A'
df = df[~df['Abstract'].isin(['N/A']) & df['Abstract'].notna()]

In [9]:
df.to_csv('6n_cleaned.csv', index=False)